In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps 
import os
from pprint import pprint

# Import API key
from api_keys import g_key #put your API keys file in the same folder as this program

In [7]:
#read in the city data csv created by WeatherPy
file = "../output_data/cityinfo.csv"
city_df = pd.read_csv(file)

In [8]:
#Configure data needed for the humidity heatmap
gmaps.configure(api_key=g_key)
locations = city_df[["City Latitude", "City Longitude"]].astype(float)
humidity = city_df["Humidity"].astype(float)
#print(locations)

In [9]:
#create and display the humidity heatmap with gmaps
fig = gmaps.figure()
humid_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(humid_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
#Narrow down the cities into ones fitting the proper conditions to serve as a vacation location
city_df.head()
test = city_df[(city_df["Max Temperature"] <= 80) & (city_df["Max Temperature"] > 50) & 
               (city_df["Cloud Coverage"] < 20) & (city_df["Humidity"] < 65) & (city_df["Humidity"] > 35) &
               (city_df["Wind Speed"] < 10)]
#We're interested in cities with max temperatures between 50 and 80 degrees, less than 20% cloud coverage, 
#humidity between 35 and 65%, and a wind speed below 10
#these conditions are pretty strict so there may be less than 10 results left at the end

#Rename a few columns to be more readable in our hotel dataframe
test =  test.rename(columns={"City Name": "City","City Country": "Country", 
                             "City Latitude": "Latitude", "City Longitude": "Longitude"}) 

hotel_df =  test[["City","Country","Latitude","Longitude"]] #create the hotel dataframe with only the necessary rows
#print(hotel_df)
hotel_df = hotel_df.head(10) #reduce the preliminary results to 10

In [11]:
#Access the api and gather the data needed to create the hotel map
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" #url to access to find hotels
hotel_names = [] #create a list to hold each hotel found
for row in hotel_df.itertuples():
    latlon = f"{row[3]},{row[4]}" #latitude and longitude for each city
    params = {
        "location": latlon, 
        "rankby": "distance", #Closest hotels first
        "type": "lodging", 
        "keyword": "hotel",
        "key": g_key,
    }
    response = requests.get(base_url, params=params).json() #access the api and find hotels
    results = response['results']
    try:
        hotel_names.append(results[0]['name']) #add the name of the closest hotel found for each city to a list
    except (KeyError, IndexError):
        hotel_names.append("N/A") #Add "N/A" to the list for any cities that do not have a suitable hotel found

hotel_df["Hotel Name"] = hotel_names
hotel_df = hotel_df[hotel_df["Hotel Name"] != "N/A"] #drop any cities that do not have a suitable hotel, as they are
#no longer a valid vacation target

In [12]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]#store hotel info in usable format
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map and display all the hotel locations with gmaps

hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure()

fig.add_layer(humid_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))